In [1]:
wd = %pwd
print('Current directory:', wd)
if wd.endswith('models'):
    %cd ..

Current directory: E:\School\ETH\BoobsProject\project\models
E:\School\ETH\BoobsProject\project


In [2]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import numpy as np
import sys, os, time, datetime

from models.base import BaseModel
from resources.flags import FLAGS, define_flags

In [3]:
# This can only be run once.
define_flags()

## Read the input data

In this case, MNIST + batch and shuffle it. In our case, it will be quite different.

In [4]:
from tensorflow.examples.tutorials.mnist import input_data

# Reset Tensorflow graph and create new session
tf.reset_default_graph()
session = tf.Session()

def shuffle(a, b):
    # Generate the permutation index array.
    permutation = np.random.permutation(a.shape[0])
    # Shuffle the arrays by giving the permutation in the square brackets.
    return a[permutation], b[permutation]

def load_data(data_dir):
    """Returns training and test tf.data.Dataset objects."""
    data = input_data.read_data_sets(data_dir, one_hot=True)
    #train_ds = tf.data.Dataset.from_tensor_slices((data.train.images,
    #                                               data.train.labels))
    #test_ds = tf.data.Dataset.from_tensors(
    #   (data.test.images, data.test.labels))
    return (data.train, data.test)

device, data_format = ('/gpu:0', 'channels_first')
#if FLAGS.no_gpu or tfe.num_gpus() <= 0:
#    device, data_format = ('/cpu:0', 'channels_last')
print('Using device %s, and data format %s.' % (device, data_format))

# Load the datasets
train_ds, test_ds = load_data(FLAGS.in_data_dir)
#train_ds = train_ds.shuffle(60000).batch(FLAGS.batch_size)
train_X, train_Y = shuffle(train_ds.images, train_ds.labels)

Using device /gpu:0, and data format channels_first.
Extracting data_in\train-images-idx3-ubyte.gz
Extracting data_in\train-labels-idx1-ubyte.gz
Extracting data_in\t10k-images-idx3-ubyte.gz
Extracting data_in\t10k-labels-idx1-ubyte.gz


In [5]:
# Setup constatnts
IMAGE_PIXELS=784
NOISE_SIZE=100
images_input = tf.placeholder(tf.float32, shape=(None, IMAGE_PIXELS))
noise_input = tf.placeholder(tf.float32, shape=(None, NOISE_SIZE))

In [6]:
## Discriminator
def discriminator(X, reuse):
    with tf.variable_scope("Discriminator", reuse=reuse):
        # Layer 1
        dx = tf.layers.dense(X, units=1024, kernel_initializer=tf.random_normal_initializer(stddev=0.02), activation=tf.nn.relu, name='fc1')
        # Layer 2
        #dx = tf.layers.dense(dx, units=512, activation=tf.nn.relu, name='fc2')
        # Layer 3
        #dx = tf.layers.dense(dx, units=256, activation=tf.nn.relu, name='fc3')
        # Layer 4
        d_out = tf.layers.dense(dx, units=1, kernel_initializer=tf.random_normal_initializer(stddev=0.02), name='fc_out')
        return d_out

In [7]:
## Generator
def generator(X, reuse=False):
    with tf.variable_scope('Generator', reuse=reuse):
        # Layer 1
        gx = tf.layers.dense(X, units=128, activation=tf.nn.relu, name='fc1')
        # Layer 2
        #gx = tf.layers.dense(gx, units=512, activation=tf.nn.relu, name='fc2')
        # Layer 3
        #gx = tf.layers.dense(gx, units=1024, activation=tf.nn.relu, name='fc3')
        # Layer 4
        g_out = tf.layers.dense(gx, units=784, activation=tf.nn.sigmoid, name='fc_out')
        return g_out

In [8]:
## Losses
g_sample = generator(noise_input)
d_real = discriminator(images_input, reuse=False)
d_fake = discriminator(g_sample, reuse=True)

d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_real, labels=tf.ones_like(d_real)))
d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_fake, labels=tf.zeros_like(d_fake)))
d_loss = d_loss_real + d_loss_fake
g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_fake, labels=tf.ones_like(d_fake)))

## Summaries
gen_image_summary_op = tf.summary.image('generated_image0', tf.reshape(g_sample[0], (1, 28, 28, 1)), max_outputs=1)
gen_image_summary_op1 = tf.summary.image('generated_image1', tf.reshape(g_sample[5], (1, 28, 28, 1)), max_outputs=1)
gen_image_summary_op2 = tf.summary.image('generated_image2', tf.reshape(g_sample[10], (1, 28, 28, 1)), max_outputs=1)
gen_image_summary_op3 = tf.summary.image('generated_image3', tf.reshape(g_sample[15], (1, 28, 28, 1)), max_outputs=1)

# Optimizers
t_vars = tf.trainable_variables()
d_opt = tf.train.AdamOptimizer(2e-4).minimize(d_loss, var_list=[var for var in t_vars if 'Discriminator' in var.name])
g_opt = tf.train.AdamOptimizer(2e-4).minimize(g_loss, var_list=[var for var in t_vars if 'Generator' in var.name])

saver = tf.train.Saver(max_to_keep=1)

In [9]:
num_test_samples=16
test_noise = np.random.uniform(-1, 1, size=(num_test_samples, NOISE_SIZE))

In [10]:
num_epochs = 200

# Start interactive session
session = tf.InteractiveSession()
# Init Variables
tf.global_variables_initializer().run()
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")
# Create writer instance and write summary
writer = tf.summary.FileWriter(logdir=FLAGS.out_data_dir + "/gan_mnist_{}".format(timestamp), graph=session.graph)

In [11]:
def next_batch(arr, batch_size):
    num_batches = int(len(arr) / batch_size)
    for i in range(0, num_batches * batch_size, batch_size):
        yield arr[i:i+batch_size]
    yield arr[num_batches*batch_size:]

# Iterate through epochs
BATCH_SIZE = 100
for epoch in range(FLAGS.epochs):
    print("Epoch %d" % epoch)
    for n_batch, batch in enumerate(next_batch(train_X, BATCH_SIZE)):
        # 1. Train Discriminator
        #X_batch = images_to_vectors(batch.permute(0, 2, 3, 1).numpy())
        batch_noise = np.random.uniform(-1, 1, [BATCH_SIZE, NOISE_SIZE]).astype(np.float32)
        feed_dict = {images_input: batch, noise_input: batch_noise}
        _, d_error, d_pred_real, d_pred_fake = session.run(
            [d_opt, d_loss, d_real, d_fake], feed_dict=feed_dict
        )

        # 2. Train Generator
        feed_dict = {noise_input: batch_noise}
        _, g_error = session.run(
            [g_opt, g_loss], feed_dict=feed_dict
        )

        if n_batch % 100 == 0:
        #    display.clear_output(True)
            # Generate images from test noise
            test_images, gen_image_summary, gen_image_summary1, gen_image_summary2, gen_image_summary3 = session.run(
                [g_sample, gen_image_summary_op, gen_image_summary_op1, gen_image_summary_op2, gen_image_summary_op3], feed_dict={noise_input: test_noise}
            )
            writer.add_summary(gen_image_summary, global_step=epoch*1000+n_batch)
            writer.add_summary(gen_image_summary1, global_step=epoch*1000+n_batch)
            writer.add_summary(gen_image_summary2, global_step=epoch*1000+n_batch)
            writer.add_summary(gen_image_summary3, global_step=epoch*1000+n_batch)
            
            print("Epoch: {}, Batch: {}, D_Loss: {}, G_Loss: {}".format(epoch, n_batch, d_error, g_error))
        #    test_images = vectors_to_images(test_images)
        #    # Log Images
        #    logger.log_images(test_images, num_test_samples, epoch, n_batch, num_batches, format='NHWC');
        #    # Log Status
        #    logger.display_status(
        #        epoch, num_epochs, n_batch, num_batches,
        #        d_error, g_error, d_pred_real, d_pred_fake
        #    )

Epoch 0
Epoch: 0, Batch: 0, D_Loss: 1.4128124713897705, G_Loss: 1.0048991441726685
Epoch: 0, Batch: 100, D_Loss: 0.01937485672533512, G_Loss: 5.944533824920654
Epoch: 0, Batch: 200, D_Loss: 0.06171085312962532, G_Loss: 7.539041519165039
Epoch: 0, Batch: 300, D_Loss: 0.11196392774581909, G_Loss: 5.143267631530762
Epoch: 0, Batch: 400, D_Loss: 0.05813948065042496, G_Loss: 4.4670915603637695
Epoch: 0, Batch: 500, D_Loss: 0.029366396367549896, G_Loss: 4.863171577453613
Epoch 1
Epoch: 1, Batch: 0, D_Loss: 0.08269477635622025, G_Loss: 4.6442060470581055
Epoch: 1, Batch: 100, D_Loss: 0.01692943647503853, G_Loss: 5.350062370300293
Epoch: 1, Batch: 200, D_Loss: 0.015572715550661087, G_Loss: 5.200153827667236
Epoch: 1, Batch: 300, D_Loss: 0.02343159168958664, G_Loss: 4.266664028167725
Epoch: 1, Batch: 400, D_Loss: 0.02806791476905346, G_Loss: 4.150373458862305
Epoch: 1, Batch: 500, D_Loss: 0.04099668562412262, G_Loss: 4.437796592712402
Epoch 2
Epoch: 2, Batch: 0, D_Loss: 0.15111324191093445, G_L

KeyboardInterrupt: 

(<tf.Tensor 'IteratorGetNext_1:0' shape=(?, 784) dtype=float32>, <tf.Tensor 'IteratorGetNext_1:1' shape=(?, 10) dtype=float64>)
